In [11]:
# IJulia.qtconsole();

# set width of notbook 
display("text/html", "<style>.container { width:80% !important; }</style>")
ENV["Columns"] = 5000;

In [52]:
using Pkg
Pkg.activate(".")
# Uncomment for first run
# Pkg.instantiate() 

Activating environment at `C:\Users\Robert\Promotion\Workshop\05_risk_trading\Project.toml`


In [13]:
using LinearAlgebra, DataFrames, Dates
using CSV
using JuMP
using Mosek, MosekTools
# using Gurobi
using Distributions

In [51]:
# Define some useful functions

function m_to_resdf(m)

    # Generator Results
    p_res = value.(m[:p])
    deltap = dual.(m[:δp])
    deltam = dual.(m[:δm])
    t_res = value.(m[:t])
    genset = collect(1:n_generators)
    gen_results_df = DataFrame(generator = genset, p_res = p_res, deltap = deltap, deltam = deltam, t = t_res)

    # Alpha Results
    alpha_res = value.(m[:α])
    alpha_results_df = DataFrame(alpha_res)
    colnames = ["alpha_u$(i)" for i in 1:n_res]
    rename!(alpha_results_df, Symbol.(colnames))
    insertcols!(alpha_results_df, 1, :generator => genset)

    # Energy 
    lambda_res = dual.(m[:λ])
    lambda_res_df = DataFrame(lambda = lambda_res)

    # Balancing
    chi_res = dual.(m[:χ])
    chi_res_df = DataFrame(chi = chi_res)


    # ADS Results
    if :μ in keys(m.obj_dict)
        mu_res = dual.(m[:μ])
    else
        mu_res = zeros(n_events)
    end
    
    if :mubal in keys(m.obj_dict)
        mu_bal_res = dual.(m[:mubal])[1,:]
    else
        mu_bal_res = zeros(Int(floor(n_events/2)))
    end
    
#     Correct forced symmetry
    mu_res = mu_res ./ 2
    mu_res[1:4] = mu_bal_res
    mu_res_df = DataFrame(mu = mu_res)

    if :a in keys(m.obj_dict)
        a_res = value.(m[:a])
    else
        a_res = zeros(n_generators, n_events)
    end
    
    ads_results_df = DataFrame(a_res)
    colnames = ["a_w$(i)" for i in 1:n_events]
    rename!(ads_results_df, Symbol.(colnames))
    insertcols!(ads_results_df, 1, :generator => genset)
    ads_payments = [sum(a_res[i,w]*mu_res[w] for w in 1:n_events) for i in 1:n_generators]
    insertcols!(ads_results_df, 2, :ads_payment => ads_payments)

    gen_res_joined_df = join(gen_results_df, alpha_results_df, on=:generator)
    gen_res_joined_df = join(gen_res_joined_df, ads_results_df, on=:generator)
    
    return gen_res_joined_df, lambda_res_df, chi_res_df, mu_res_df
end


function save_results(res_dfs, ex_name)
    timestamp = Dates.format(Dates.now(), "yymmdd_HHMM")
    resultdf_names = ["gen_results", "lambda", "chi", "mu"]
    savepath = "results/$(ex_name)_$(timestamp)"
    for (name, res_df) in res_dfs
        mkpath("$(savepath)/$(name)")
        for (i,res) in enumerate(resultdf_names)
            CSV.write("$(savepath)/$(name)/$(res).csv", res_df[i])
        end
    end   
    
    timestamp = "latest"
    resultdf_names = ["gen_results", "lambda", "chi", "mu"]
    savepath = "results/$(ex_name)_$(timestamp)"
    for (name, res_df) in res_dfs
        mkpath("$(savepath)/$(name)")
        for (i,res) in enumerate(resultdf_names)
            CSV.write("$(savepath)/$(name)/$(res).csv", res_df[i])
        end
    end   
end




save_results (generic function with 1 method)

In [45]:
# Data Generation

# Generators
n_generators = 5

c=zeros(2,n_generators)
c[1,1] = 10
c[1,2] = 7
c[1,3] = 7
c[1,4] = 15
c[1,5] = 17

c[2,:] = 0.1 .* c[1,:]

pmax = ones(n_generators)

pmax[1] = 30
pmax[2] = 10
pmax[3] = 10
pmax[4] = 25
pmax[5] = 25

# Demand
D = 100

# RES
n_res = 5
res = zeros(n_res)
res[1] = 5
res[2] = 5
res[3] = 5
res[4] = 5
res[5] = 5

σ_true = 0.2 .* res

Σ_true = diagm(0 => σ_true)

5×5 Array{Float64,2}:
 1.0  0.0  0.0  0.0  0.0
 0.0  1.0  0.0  0.0  0.0
 0.0  0.0  1.0  0.0  0.0
 0.0  0.0  0.0  1.0  0.0
 0.0  0.0  0.0  0.0  1.0

In [46]:
# Create some uncertainty sets
K_total = 50-1
max_correlation = 0.5
σ_max = 2 .* σ_true
Σ_k = [Σ_true]
for k in 1:K_total
    im_flag = true
    tries = 0
    while im_flag && (tries <=50)
        global Sk = zeros(n_res,n_res)
        sigmas = [σ_max[i] * rand() for i in 1:n_generators]
        corr = zeros(n_res,n_res)
        for i in 1:n_res
            for j in i:n_res
                if i == j
                    corr = 1
                else
                    corr = rand()*max_correlation # Negative Correlation in RES doesnät really make sense
                end
                Sk[i,j] = corr*sigmas[i]*sigmas[j]
                Sk[j,i] = corr*sigmas[i]*sigmas[j]
            end
        end
        imval = sum(imag(sqrt(Sk)))
        if imval == 0
            im_flag = false
        end
        tries = tries + 1
    end
    if tries == 51
        println("WARNING, non rootable covarmat at $k")
    end
    push!(Σ_k, Sk)
end

# Every Producers shares the true and one random covariance matrix
# 8 more a picked randomly from the available matrices

In [47]:
K = 10
K_set = []
shared = [1]
for i in 1:n_generators
    K_set_i = vcat(shared, Int.(ceil.(rand(K - length(shared)).*(K_total-2) .+ 2)))
    push!(K_set, K_set_i)
end


# Calculate ADS events
n_events = 8
res_forecast = sum(res)
W = []
push!(W, Dict("l" => -99999., "u" => -0.2*res_forecast))
push!(W, Dict("l" => W[1]["u"], "u" => -0.1*res_forecast))
push!(W, Dict("l" => W[2]["u"], "u" => -0.05*res_forecast))
push!(W, Dict("l" => W[3]["u"], "u" => 0.))
push!(W, Dict("l" => W[4]["u"], "u" => 0.05*res_forecast))
push!(W, Dict("l" => W[5]["u"], "u" => 0.1*res_forecast))
push!(W, Dict("l" => W[6]["u"], "u" => 0.2*res_forecast))
push!(W, Dict("l" => W[7]["u"], "u" => 99999.))

# Calculate Event Probabilities
P = zeros(n_events, K_total) 
for (w, event) in enumerate(W)
    for k in 1:K_total
        sigma_k = sqrt(sum(Σ_k[k]))
        d_k = Normal(0, sigma_k)
        p_up = cdf(d_k, event["u"])
        p_low = cdf(d_k, event["l"])
        P[w,k] = p_up-p_low
    end
end

In [48]:
# Simple ED Model WITH risk-trading

m_ads = Model(Mosek.Optimizer)
set_optimizer_attribute(m_ads, "MSK_IPAR_LOG", 0)

@variable(m_ads, p[1:n_generators])
@variable(m_ads, α[1:n_generators, 1:n_res] >= 0)
@variable(m_ads, a[1:n_generators, 1:n_events])
@variable(m_ads, t[1:n_generators]) # Auxillary WC expected cost
@variable(m_ads, s[1:n_generators, 1:K] >= 0) # Auxiallary Quadratic Balancing Cost

@objective(m_ads, Min, sum(c[2,i]*p[i]^2 + c[1,i]*p[i] + t[i] for i in 1:n_generators))

# Expected cost
bal_part = []

for k in 1:K
    for i in 1:n_generators
        Σ_k_root = sqrt(Σ_k[K_set[i][k]])
        @constraint(m_ads, vec(hcat(s[i,k], 0.5, α[i,:]'*Σ_k_root)) in  RotatedSecondOrderCone())
    end
end
@constraint(m_ads, eta[i=1:n_generators, k=1:K], t[i] >= c[2,i]*s[i,k] - sum(a[i,w]*(P[:,K_set[i]][w,k]) for w in 1:n_events))

# Generator Limits
@constraint(m_ads, δp[i=1:n_generators], p[i] <= pmax[i])
@constraint(m_ads, δm[i=1:n_generators], p[i] >= 0)

# Energy Balance
@constraint(m_ads, λ, sum(p[i] for i in 1:n_generators) == D - sum(res))

# Reserve Balance
@constraint(m_ads, χ[u=1:n_res], sum(α[i,u] for i in 1:n_generators) == 1.)

# ADS Balance
@constraint(m_ads, μ[w=1:n_events], sum(a[i,w] for i in 1:n_generators) == 0.)
@constraint(m_ads, mubal[i=1:n_generators, w=1:4], a[i,w] == a[i,n_events-w+1]) # Because of symmetry solver would cluster decisions on one side only

println("ok")
optimize!(m_ads)

ok


In [49]:
# Very Simple ED Model WITHOUT risk-trading

m = Model(Mosek.Optimizer)
set_optimizer_attribute(m, "MSK_IPAR_LOG", 0)

@variable(m, p[1:n_generators])
@variable(m, α[1:n_generators, 1:n_res] >= 0)
@variable(m, t[1:n_generators]) # Auxillary WC expected cost
@variable(m, s[1:n_generators, 1:K] >= 0) # Auxiallary Quadratic Balancing Cost

@objective(m, Min, sum(c[2,i]*p[i]^2 + c[1,i]*p[i] + t[i] for i in 1:n_generators))

# Expected cost
bal_part = []

for k in 1:K
    for i in 1:n_generators
        Σ_k_root = sqrt(Σ_k[K_set[i][k]])
        @constraint(m, vec(hcat(s[i,k], 0.5, α[i,:]'*Σ_k_root)) in  RotatedSecondOrderCone())
    end
end
@constraint(m, eta[i=1:n_generators, k=1:K], t[i] >= c[2,i]*s[i,k])

# Generator Limits
@constraint(m, δp[i=1:n_generators], p[i] <= pmax[i])
@constraint(m, δm[i=1:n_generators], p[i] >= 0)

# Energy Balance
@constraint(m, λ, sum(p[i] for i in 1:n_generators) == D - sum(res))

# Reserve Balance
@constraint(m, χ[u=1:n_res], sum(α[i,u] for i in 1:n_generators) == 1.)


println("ok")
optimize!(m)

ok


In [50]:
results_dfs = Dict(
    "withADS" => m_to_resdf(m_ads),
    "noADS" => m_to_resdf(m)
)

Ps = P[:,K_set[1]]
for i in 2:n_generators
    Ps = hcat(Ps, P[:,K_set[i]])
end
Ps_df = DataFrame(Ps');

save_results(results_dfs, "simple_ED")
CSV.write("results/Ps.csv", Ps_df)

"results/Ps.csv"

In [56]:
results_dfs["withADS"][1]

,generator,p_res,deltap,deltam,t,alpha_u1,alpha_u2,alpha_u3,alpha_u4,alpha_u5,ads_payment,a_w1,a_w2,a_w3,a_w4,a_w5,a_w6,a_w7,a_w8
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,26.0435,-4.98694e-10,-0.0,0.718262,0.185816,0.191672,0.230615,0.307363,0.239271,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
2,2,10.0,-41.087,-2.3867e-10,0.782135,0.385379,0.256744,0.31653,0.10113,0.250526,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
3,3,10.0,-41.087,-2.3867e-10,0.769047,0.367407,0.274478,0.120562,0.284919,0.227279,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
4,4,15.6957,1.99063e-10,-4.23984e-10,0.452166,0.0382066,0.148926,0.179412,0.138001,0.204543,0.0,-0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,0.0
5,5,13.2609,3.09217e-10,-3.67258e-10,0.362636,0.0231907,0.12818,0.152881,0.168587,0.0783809,0.0,0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0


In [37]:
results_dfs["noADS"][1]

,generator,p_res,deltap,deltam,t,alpha_u1,alpha_u2,alpha_u3,alpha_u4,alpha_u5,ads_payment,a_w1,a_w2,a_w3,a_w4,a_w5,a_w6,a_w7,a_w8
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,26.0435,-4.98694e-10,-0.0,0.718262,0.185816,0.191672,0.230615,0.307363,0.239271,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10.0,-41.087,-2.3867e-10,0.782135,0.385379,0.256744,0.31653,0.10113,0.250526,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,10.0,-41.087,-2.3867e-10,0.769047,0.367407,0.274478,0.120562,0.284919,0.227279,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,15.6957,1.99063e-10,-4.23984e-10,0.452166,0.0382066,0.148926,0.179412,0.138001,0.204543,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,13.2609,3.09217e-10,-3.67258e-10,0.362636,0.0231907,0.12818,0.152881,0.168587,0.0783809,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [21]:
objective_value(m_ads)

2350.805554211267

In [22]:
objective_value(m)

2351.127725563135

In [28]:
results_dfs["withADS"][2]

,lambda
,Float64
1,62.087


In [29]:
results_dfs["noADS"][2]

,lambda
,Float64
1,62.087


In [30]:
results_dfs["withADS"][3]

,chi
,Float64
1,1.24509
2,1.54053
3,0.728824
4,0.694018
5,1.31584


In [31]:
results_dfs["noADS"][3]

,chi
,Float64
1,0.918474
2,1.47972
3,1.31042
4,1.3431
5,1.11676


In [62]:
dual.(m_ads[:mubal])[1,:]

4-element Array{Float64,1}:
 -0.09919869291538723
 -0.1598964806730103 
 -0.1140495888549063 
 -0.12685523659609246

In [61]:
dual.(m_ads[:μ]) ./ 2

8-element Array{Float64,1}:
  0.0                
  0.0                
  0.0                
  0.0                
 -0.12685523659609246
 -0.11404958549086901
 -0.15989648055635408
 -0.09919869137333921